In [1]:
import pandas as pd

df_eq1 = pd.read_csv("significant_earthquakes1965-2016.csv", sep=",")
df_eq2 = pd.read_csv("global_significant_earthquake_db_from_2150BC.csv", sep=",")
df_eq2_after_1900 = df_eq2[df_eq2["YEAR"] >= 1900]
df_eq3 = pd.read_csv("emdat_public_2021_10_07_query_uid-91b8Mu.csv", sep=",", skiprows=6)

In [2]:
class Earthquake:
    
    def __init__(self,year,month,day,long,lat):
    
        self.year = year
        self.month = month
        self.day = day
        self.long = long
        self.lat = lat
        self.country = None
        self.id = None
    
    def add_country(self,country):
        self.country = country
        
    
    def add_id(self,id):
        self.id = id
    
    def compare_earthquake(self,e2):
        if e2.year == self.year:
            if e2.month == self.month:
                if e2.day == self.day:
                    try:
                        self.long = round(float(self.long))
                        self.lat = round(float(self.lat))
                    except Exception:
                        return "missing long/lat"
                    try:
                        e2.long = round(float(e2.long))
                        e2.lat = round(float(e2.lat))
                        if e2.long == self.long:
                            if e2.lat == self.lat:
                                return "yes"
                            else:
                                return "partial"
                        else:
                            return "no"
                    except Exception:
                        return "missing long/lat"
        return "no"

In [3]:
# create Earthquake objects from datasets

earthquakes1 = []
earthquakes2 = []
earthquakes3 = []


# eq1
for index, row in df_eq1.iterrows():
    date = row["Date"].split("/")
    if len(date) == 3:
        e = Earthquake(int(date[2]), int(date[0].lstrip('0')), int(date[1].lstrip('0')), row["Longitude"], row["Latitude"])
        earthquakes1.append(e)
        
# eq2
for index, row in df_eq2_after_1900.iterrows():
    e = Earthquake(row["YEAR"], row["MONTH"], row["DAY"], row["LONGITUDE"], row["LATITUDE"])
    earthquakes2.append(e)

# eq3
i = 1
for index, row in df_eq3.iloc[8:].iterrows():
    e = Earthquake(row["Start Year"], row["Start Month"], row["Start Day"], row["Longitude"], row["Latitude"])
    e.add_country(row["ISO"])
    e.add_id(i)
    earthquakes3.append(e)
    i += 1

In [7]:
# detect earthquakes on same day in same location for dataset3
all_matches = {}
for e1 in earthquakes3:
    matches = []
    for e2 in earthquakes3:
        if e1.id != e2.id:
            if e1.country == e2.country:
                if e1.year == e2.year:
                    if e1.month == e2.month:
                        if e1.day == e2.day:
                            matches.append(e2.id)
    if len(matches) > 0:
        all_matches[e1.id] = matches
print(all_matches)

{129: [1531], 283: [1494], 310: [1364], 1364: [310], 1374: [1375], 1375: [1374], 1494: [283], 1531: [129]}


In [ ]:
matching_earthquakes = set()
for k,v in all_matches.items():
    matching_earthquakes.add(())

In [ ]:
# compute overlap between eq1 and [eq2 & eq3]
matches1 = 0
missing_values1 = 0

e2_match = 0
e3_match = 0
triple_match = 0
for e1 in earthquakes1:
    matches_eq2 = False
    matches_eq3 = False
    for e2 in earthquakes2:
        com2 = e1.compare_earthquake(e2)
        if com2 == "yes":
            matches_eq2 = True
            matches1 +=1
            break
        if com2 == "missing long/lat":
            #matches_eq2 = True
            missing_values1 += 1
            break
    for e3 in earthquakes3:
        com3 = e1.compare_earthquake(e3)
        if com3 == "yes":
            matches_eq3 = True
            matches1 +=1
            break
        if com3 == "missing long/lat":
            #matches_eq3 = True
            missing_values1 += 1
            break
    if matches_eq2 and matches_eq3:
        triple_match += 1
    elif matches_eq2:
        e2_match += 1
    elif matches_eq3:
        e3_match += 1

print(e2_match)
print(e3_match)
print(triple_match)

In [ ]:
# compute overlap between eq2 and eq3
matches = 0
missing_values = 0
for e2 in earthquakes2:
    for e3 in earthquakes3:
        com = e2.compare_earthquake(e3)
        if com == "yes":
            matches +=1
            break
        if com == "missing long/lat":
            missing_values += 1
            break

print(matches)
print(missing_values)